In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install python3-wget

In [ ]:
pip install pycurl

In [ ]:
pip install selenium

In [ ]:
pip install html5lib

In [ ]:
pip install webdriver-manager

In [ ]:
pip install uuid

In [ ]:
pip install psycopg2

In [ ]:
# STEP 0: DATABASE TABLES CREATION.

In [ ]:
CREATE EXTENSION pgcrypto;


--
-- Name: pub_review_paragraphs; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.pub_review_paragraphs (
    id uuid DEFAULT public.gen_random_uuid() NOT NULL,
    pub_id uuid NOT NULL,
    paragraph_text text NOT NULL,
    date_created timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    date_updated timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    username text NOT NULL
);


ALTER TABLE public.pub_review_paragraphs OWNER TO postgres;

--
-- Name: publication_urls; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.publication_urls (
    id uuid DEFAULT public.gen_random_uuid() NOT NULL,
    site_id uuid,
    date_created timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    date_updated timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    username text NOT NULL,
    pub_url text NOT NULL,
    is_scraped boolean DEFAULT false NOT NULL
);


ALTER TABLE public.publication_urls OWNER TO postgres;

--
-- Name: site_review_urls; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.site_review_urls (
    id uuid DEFAULT public.gen_random_uuid() NOT NULL,
    site_url text NOT NULL,
    date_created timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    date_updated timestamp with time zone DEFAULT CURRENT_TIMESTAMP NOT NULL,
    username text NOT NULL
);


ALTER TABLE public.site_review_urls OWNER TO postgres;

--
-- Name: pub_review_paragraphs pub_review_paragraphs_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.pub_review_paragraphs
    ADD CONSTRAINT pub_review_paragraphs_pkey PRIMARY KEY (id);


--
-- Name: publication_urls publication_urls_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.publication_urls
    ADD CONSTRAINT publication_urls_pkey PRIMARY KEY (id);


--
-- Name: site_review_urls site_review_urls_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.site_review_urls
    ADD CONSTRAINT site_review_urls_pkey PRIMARY KEY (id);


--
-- Name: pub_review_paragraphs fk_puburl_pubid; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.pub_review_paragraphs
    ADD CONSTRAINT fk_puburl_pubid FOREIGN KEY (pub_id) REFERENCES public.publication_urls(id);


--
-- Name: publication_urls fk_puburl_siteid; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.publication_urls
    ADD CONSTRAINT fk_puburl_siteid FOREIGN KEY (site_id) REFERENCES public.site_review_urls(id);



https://starkandwayne.com/blog/uuid-primary-keys-in-postgresql/   

In [ ]:
import psycopg2
# method to get a connection to local PostgreSQL database
def get_conn_rds(): 

    try:
        # connect to the postgresql database
        print("Connecting to the PostgreSQL database...")

        connection = psycopg2.connect(user = "postgres",
                                  password = "9UhLiYiX",   
                                  host = "twiviews.cco0dsf0nmqk.us-east-2.rds.amazonaws.com",         # and hostname
                                  port = "5432",
                                  database = "twiviews")

        return connection

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
        
conn = get_conn_rds()
print(conn)

In [ ]:
import uuid
import datetime

## Insert into table : site_review_urls. 
## date_created, date_updated are set to default CURRENT_TIMESTAMP. No need to pass them in inserts anymore.

def insert_to_tbl_site_review_urls(site_url, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        
        # please note the change here. we longer need to pass date_created, date_updated. I have now set these
        # to default value CURRENT_TIMESTAMP for inserts. 
        query_params_list = {'site_url': site_url,                              
                             'username': username}
        sql = '''
                INSERT INTO public."site_review_urls"(site_url, username)
                        VALUES (%(site_url)s, %(username)s);
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully inserted a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()  

In [ ]:
# STEP 1 - You get urls for all movies on page1 page2 and so on.

In [ ]:
from selenium import webdriver
import uuid
import psycopg2

links_page1 = []
result_urls = []
options = webdriver.ChromeOptions()
options.add_argument('headless')
site_movies_by_year_desc_url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?sort=desc'

# webscraper method to first get all the urls on the above url where I'm filtering "desc" for current year 2020.
# similarly, the above url can be changed to the year 2019. 
# The following method just scrapes all the review urls on page 1. I'll need to do the same for page2 and so on.
def fetchandinsert_movie_releases_by_year_urls():
    browser = webdriver.Chrome("/home/srini/pyprjs/scrpr/chromedriver", options=options)
    browser.get(site_movies_by_year_desc_url)
    css_selector_metacritic_review_links = "div.clamp-metascore"
    divs = browser.find_elements_by_css_selector(css_selector_metacritic_review_links)
    for div in divs:
        site_url = div.find_element_by_css_selector('a').get_attribute('href')
        # inserts the scraped urls on the page into the database table :metacritics_review_urls
        insert_to_tbl_site_review_urls(site_url, 'Srini')
        links_page1.append(site_url)
    return links_page1

result_urls = fetchandinsert_movie_releases_by_year_urls()
print(len(result_urls))

In [ ]:
# STEP 2 - read the urls from the database and now for each of these urls - we'll need to get publication urls for 
#          critic reviews.

In [ ]:
## 2.a SELECT URLS FROM DATABASE TABLE: metacritic_review_urls

In [ ]:
import psycopg2


def select_from_tbl_site_review_urls():
    
    site_review_urls = dict()
    
    try:
        conn = get_conn_rds()
        cursor = conn.cursor()
        select_query = "select * from public.site_review_urls"
        cursor.execute(select_query)
        print("Selecting rows from the table using cursor.fetchall")
        result_set = cursor.fetchall() 
   
        for row in result_set:
            site_review_urls[row[0]]= row[1]
            
       
    except (Exception, psycopg2.Error) as error :
        print ("Error while fetching data from PostgreSQL", error)

    finally:
    #closing database connection.
        if(conn):
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")
    
    return site_review_urls


result_dict = select_from_tbl_site_review_urls()
for key, value in result_dict.items():
    print(f"{key} : {value}")

In [ ]:
## 2.b INSERT INTO TABLE: publication_urls to be used by: fetchandinsert_publication_urls_per_metacritic_url. 
##     Please see 2.c.
## date_created, date_updated are set to default CURRENT_TIMESTAMP. No need to pass them in inserts anymore.

In [ ]:
import uuid
import datetime

def insert_to_tbl_publication_urls(site_id, pub_url, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        query_params_list = {'site_id' : site_id,
                             'pub_url': pub_url,                              
                             'username': username}
        sql = '''
                INSERT INTO public."publication_urls"(site_id, pub_url, username)
                        VALUES (%(site_id)s, %(pub_url)s, %(username)s);
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully inserted a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
## STPE 3 - Method to fetch the publication urls per metacritic review url.
##     Makes use of methods: 2.a and 2.b.
##     Please note the time.sleep - we'll need to pause a while to not be aggressive.
##     If scraping night time, these could be relaxed, say commenting inner for loop sleep 
##     and changing outer for loop sleep to 60 seconds.

In [ ]:
import time
pub_links =[]
options = webdriver.ChromeOptions()
options.add_argument('headless')
def fetchandinsert_publication_urls_per_site_url():
    
    site_review_urls = select_from_tbl_site_review_urls()  
        
    browser = webdriver.Chrome("/home/srini/pyprjs/scrpr/chromedriver", options=options)
    for site_id, site_url in site_review_urls.items():
            browser.get(site_url)
            webElem = browser.find_elements_by_css_selector("a.read_full")
            for elem in webElem:
                pub_url = elem.get_attribute('href')
                insert_to_tbl_publication_urls(site_id, pub_url, 'Srini')
                pub_links.append(pub_href)
                time.sleep(60)
            time.sleep(120)
        
    
fetchandinsert_publication_urls_per_site_url()        

In [ ]:
# The below two files are the csv exports for review of the data from above two tables: 
# metacritic_review_urls,
# publication_urls 

In [ ]:
!ls -lrth *.csv

In [ ]:
## Utility methods for STEP 4
## 4.a

def select_un_scraped_publication_urls():
    
    pub_urls = dict()
    
    try:
        conn = get_conn_rds()
        cursor = conn.cursor()
        select_query = "select * from public.publication_urls where is_scraped = 'False'"
        cursor.execute(select_query)
        print("Selecting is_scraped false rows from the table using cursor.fetchall")
        result_set = cursor.fetchall() 
   
        for row in result_set:
            pub_urls[row[0]]= row[5]
            
       
    except (Exception, psycopg2.Error) as error :
        print ("Error while fetching data from PostgreSQL", error)

    finally:
    #closing database connection.
        if(conn):
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")
    
    return pub_urls


result_dict = select_un_scraped_publication_urls()
for key, value in result_dict.items():
    print(f"{key} : {value}")
    break

In [ ]:
## Utility methods for STEP 4
## 4.b
## date_created, date_updated are set to default CURRENT_TIMESTAMP. No need to pass them in inserts anymore.

def insert_pub_review_paragraphs(pub_id, pub_text, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        query_params_list = {'pub_id' : pub_id,
                             'paragraph_text': pub_text,                             
                             'username': username}
        sql = '''
                INSERT INTO public."pub_review_paragraphs"(pub_id, paragraph_text, username)
                        VALUES (%(pub_id)s, %(paragraph_text)s, %(username)s);
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully inserted a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()    

In [ ]:
## No need to pass in date_updated while UPDATE command. A trigger is setup for three tables.
## so now you don't have to always remember to update the time stamps. You can just let PostgreSQL handle it for you.

def set_publication_urls_is_scraped(pub_id, is_scraped, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        query_params_list = {'pub_id' : pub_id,   
                             'is_scraped': is_scraped,                                                      
                             'username': username}
        sql = '''
                UPDATE public."publication_urls"
                        set is_scraped = %(is_scraped)s, username= %(username)s where id = %(pub_id)s;
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully updated a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
# STEP 4 - tested it out on one url. And this method will change to fetchall from publication_urls table
#          and extract <p> tags for paragraphs on each of the pub_url,
#          i.e publication site and the extracted paragraphs will be inserted into the new database table: 
#          pub_review_paragraphs. 
#          After all the pub_urls for specific pub_id (id from publication_urls table) is scraped, we set is_scraped to True within publication_urls - to track completion.
#          We don't need to track is_scraped on site_review_urls table as we're only interested in the 
#          count for publication urls scraped for tracking purpose. just as a stat for training data.
#          Process goes on until all are completed.

In [18]:
from bs4 import BeautifulSoup
import requests
import time
def pub_links_scrape_review():
    pubid_puburls = select_un_scraped_publication_urls() 
    for pub_id, pub_url in pubid_puburls.items():
        is_scraped = False
        try:
            response = requests.get(pub_url)
            html = response.text
            soup = BeautifulSoup(html, "html5lib")
            for ptag in soup.find_all('p'):
                if len(ptag.text) > 100:  #adding paragraph texts that atleast have 100 chars.
                    insert_pub_review_paragraphs(pub_id, ptag.text, 'Srini')
            is_scraped = True                        
        except:
            print("pub_url errored out: "+pub_url+" and is_scraped",is_scraped)
        set_publication_urls_is_scraped(pub_id, is_scraped, 'Srini')
        time.sleep(30)        

        
pub_links_scrape_review()    

Connecting to the PostgreSQL database...
Selecting is_scraped false rows from the table using cursor.fetchall
PostgreSQL connection is closed
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL database...
Successfully inserted a record
Closing Connection.
Connecting to the PostgreSQL d

NameError: name 'time' is not defined